In [ ]:
# import libraries
import random
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from shapely.geometry import Point
from shapely.ops import unary_union
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
import FrameworkLib as FL
import Plot as PL
import numpy as np

In [ ]:
# Define the box margins
box_margin = 10


# maximum distance between agents
max_dist = 7

# min and max agents
min_agents = 8
max_agents = 8

# num of graphs for each number of agents
num_graphs = 2000

graphs = []

# Generate graphs
for num_agents in range(min_agents, max_agents + 1):
    num_graph = 0
    while num_graph < num_graphs:
        
        # Assign random positions within a box (e.g., 10x10)
        pos = {i: (random.uniform(0, box_margin), random.uniform(0, box_margin)) for i in range(num_agents)}
        
        # Generate a random graph with num_agents nodes
        G = FL.create_graph()
        
        # add nodes with the node attribute "pos"
        for i in range(num_agents):
            FL.add_node(G, i, pos=pos[i])
        
        # Add edges only if the nodes are within max_dist
        for i in range(num_agents):
            for j in range(i + 1, num_agents):
                if np.linalg.norm(np.array(pos[i]) - np.array(pos[j])) < max_dist:
                    FL.add_edge(G, i, j)
                    
        # if the graph is connected, add it to the list
        # if FL.is_rigid(G):
        if nx.is_connected(G):
        # if True:
            graphs.append(G)
            num_graph += 1

In [ ]:
spreadiness_x = []
spreadiness_y = []
EGVL_RIG = []

for G in graphs:
    # Compute the Laplacian matrix
    L = FL.get_laplacian_matrix(G)
    
    # get the x,y pos of all the nodes in separate vectors
    pos = nx.get_node_attributes(G, 'pos')
    x = np.asarray([pos[i][0] for i in range(len(pos))])
    y = np.asarray([pos[i][1] for i in range(len(pos))])
    
    # get spreadiness on x and y using laplacian
    spread_x = np.sqrt(np.dot(x, np.dot(L, x.transpose())))
    spread_y = np.sqrt(np.dot(y, np.dot(L, y.transpose())))
    
    # get the rigidity matrix
    R, RR, EGVL, EGVT = FL.get_rigidity_matrix(G)
    EGVL_rig = np.real(EGVL[3])
    
    # get the edge connectivity
    edge_connectivity = FL.get_edge_connectivity(G)
    
    # get vertex connectivity
    vertex_connectivity = FL.get_vertex_connectivity(G)
    
    # get algebraic connectivity
    algebraic_connectivity = FL.get_algebraic_connectivity(G)
    
    # Save values as graph attributes
    G.graph['spreadiness_x'] = spread_x
    G.graph['spreadiness_y'] = spread_y
    G.graph['EGVL_RIG'] = EGVL_rig
    G.graph['edge_connectivity'] = edge_connectivity
    G.graph['vertex_connectivity'] = vertex_connectivity
    G.graph['algebraic_connectivity'] = algebraic_connectivity
    
# filter all the graphs that are not rigid
rigid_graphs = [G for G in graphs if G.graph['EGVL_RIG'] > 1e-2]
    

In [ ]:
# Define the maximum row length
max_row_length = 3

# Filter graphs based on the number of nodes
filtered_graphs = {n: [] for n in range(min_agents, max_agents + 1)}
for G in graphs:
    num_nodes = len(G.nodes)
    if num_nodes in filtered_graphs:
        filtered_graphs[num_nodes].append(G)

# Calculate the number of rows needed
num_rows = (max_agents - min_agents + 1 + max_row_length - 1) // max_row_length

# Create a subplot grid
fig, axes = plt.subplots(nrows=num_rows, ncols=max_row_length, figsize=(5 * max_row_length, 5 * num_rows))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Plot each graph's spreadiness_x and EGVL_RIG in the corresponding subplot
for idx, num_nodes in enumerate(range(min_agents, max_agents + 1)):
    ax = axes[idx]
    spreadiness_x = [G.graph['spreadiness_x'] for G in filtered_graphs[num_nodes]]
    spreadiness_y = [G.graph['spreadiness_y'] for G in filtered_graphs[num_nodes]]
    spreadiness_norm = [np.sqrt(spread_x**2 + spread_y**2) for spread_x, spread_y in zip(spreadiness_x, spreadiness_y)]
    EGVL_RIG = [G.graph['EGVL_RIG'] for G in filtered_graphs[num_nodes]]
    
    # do the same for the non rigid graphs
    spreadiness_x_rigid = [G.graph['spreadiness_x'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    spreadiness_y_rigid = [G.graph['spreadiness_y'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    spreadiness_norm_rigid = [np.sqrt(spread_x**2 + spread_y**2) for spread_x, spread_y in zip(spreadiness_x_rigid, spreadiness_y_rigid)]
    EGVL_RIG_rigid = [G.graph['EGVL_RIG'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    
    ax.scatter(spreadiness_norm, EGVL_RIG, label='Non Rigid')
    ax.scatter(spreadiness_norm_rigid, EGVL_RIG_rigid, label='Rigid')
    ax.set_title(f'Number of Nodes: {num_nodes}')
    ax.set_xlabel('Spreadiness')
    ax.set_ylabel('EGVL RIG')
    ax.legend()

# Hide any unused subplots
for idx in range(len(range(min_agents, max_agents + 1)), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()


In [ ]:
# Define the maximum row length
max_row_length = 3

# Filter graphs based on the number of nodes
filtered_graphs = {n: [] for n in range(min_agents, max_agents + 1)}
for G in graphs:
    num_nodes = len(G.nodes)
    if num_nodes in filtered_graphs:
        filtered_graphs[num_nodes].append(G)

# Calculate the number of rows needed
num_rows = (max_agents - min_agents + 1 + max_row_length - 1) // max_row_length

# Create a subplot grid
fig, axes = plt.subplots(nrows=num_rows, ncols=max_row_length, figsize=(5 * max_row_length, 5 * num_rows))
# Flatten the axes array for easy indexing
axes = axes.flatten()

# Plot each graph's algebraic connectivity and vertex connectivity in the corresponding subplot
for idx, num_nodes in enumerate(range(min_agents, max_agents + 1)):
    
    # Get the current axis
    ax = axes[idx]
    graph_ids = list(range(1, num_graphs + 1))

    # Sort the graphs based on the number of edges
    sorted_graphs = sorted(filtered_graphs[num_nodes], key=lambda g: g.number_of_edges())

    # Update the graph_ids, algebraic_connectivity, vertex_connectivity, and edge_relation lists based on the sorted graphs
    graph_ids = list(range(1, len(sorted_graphs) + 1))
    algebraic_connectivity = [G.graph['algebraic_connectivity'] for G in sorted_graphs]
    vertex_connectivity = [G.graph['vertex_connectivity'] for G in sorted_graphs]
    edge_connectivity = [G.graph['edge_connectivity'] for G in sorted_graphs]
    edge_relation = 2 * np.asarray(edge_connectivity) * (1 - np.cos(np.pi / num_nodes))
    
    # ax.plot(graph_ids, algebraic_connectivity, label='Algebraic Connectivity', color='blue')
    # ax.plot(graph_ids, edge_relation, label='Edge Connectivity Relation', color='red')
    # ax.plot(graph_ids, algebraic_connectivity - edge_relation, label='a(G) - f(e(G))', color='black')
    
    # same thing with rigid graphs
    graph_ids_rigid = list(range(1, len(rigid_graphs) + 1))
    algebraic_connectivity_rigid = [G.graph['algebraic_connectivity'] for G in rigid_graphs]
    vertex_connectivity_rigid = [G.graph['vertex_connectivity'] for G in rigid_graphs]
    edge_connectivity_rigid = [G.graph['edge_connectivity'] for G in rigid_graphs]
    edge_relation_rigid = 2 * np.asarray(edge_connectivity_rigid) * (1 - np.cos(np.pi / num_nodes))
    
    ax.plot(graph_ids_rigid, algebraic_connectivity_rigid, label='Algebraic Connectivity Rigid', color='green')
    
    ax.set_title(f'Number of Nodes: {num_nodes}')
    ax.set_xlabel('Graph ID - sorted with num of edges')
    ax.set_ylabel('Connectivity')
    ax.legend()

# Hide any unused subplots
for idx in range(len(range(min_agents, max_agents + 1)), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Define the maximum row length
max_row_length = 3

# Calculate the number of rows needed
num_rows = (max_agents - min_agents + 1 + max_row_length - 1) // max_row_length

# Create a subplot grid
fig, axes = plt.subplots(nrows=num_rows, ncols=max_row_length, figsize=(5 * max_row_length, 5 * num_rows))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Plot each graph's edge relation and EGVL_RIG in the corresponding subplot
for idx, num_nodes in enumerate(range(min_agents, max_agents + 1)):
    
    ax = axes[idx]
    
    edge_relation = [2 * G.graph['edge_connectivity'] * (1 - np.cos(np.pi / num_nodes)) for G in filtered_graphs[num_nodes]]
    algebraic_connectivity = [G.graph['algebraic_connectivity'] for G in filtered_graphs[num_nodes]]
    EGVL_RIG = [G.graph['EGVL_RIG'] for G in filtered_graphs[num_nodes]]
    # ax.scatter(edge_relation, EGVL_RIG, color='blue', label='Edge Relation')
    ax.scatter(algebraic_connectivity, EGVL_RIG, color='red', label='Algebraic Connectivity')
    
    # same thing with rigid graphs
    edge_relation_rigid = [2 * G.graph['edge_connectivity'] * (1 - np.cos(np.pi / num_nodes)) for G in rigid_graphs if len(G.nodes) == num_nodes]
    algebraic_connectivity_rigid = [G.graph['algebraic_connectivity'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    EGVL_RIG_rigid = [G.graph['EGVL_RIG'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    ax.scatter(algebraic_connectivity_rigid, EGVL_RIG_rigid, color='green', label='Algebraic Connectivity Rigid')
    
    ax.set_title(f'Number of Nodes: {num_nodes}')
    ax.set_xlabel('Connectivity metrics')
    ax.set_ylabel('EGVL RIG')
    ax.legend()

# Hide any unused subplots
for idx in range(len(range(min_agents, max_agents + 1)), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Define the maximum row length
max_row_length = 3

# Calculate the number of rows needed
num_rows = (max_agents - min_agents + 1 + max_row_length - 1) // max_row_length

# Create a subplot grid
fig, axes = plt.subplots(nrows=num_rows, ncols=max_row_length, figsize=(5 * max_row_length, 5 * num_rows))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Define the threshold for EGVL_RIG
threshold = max([G.graph['EGVL_RIG'] for G in graphs]) / 2

# Plot each graph's edge relation and spreadiness in the corresponding subplot
for idx, num_nodes in enumerate(range(min_agents, max_agents + 1)):
    ax = axes[idx]
    edge_relation = [2 * G.graph['edge_connectivity'] * (1 - np.cos(np.pi / num_nodes)) for G in filtered_graphs[num_nodes]]
    spreadiness_norm = [np.sqrt(G.graph['spreadiness_x']**2 + G.graph['spreadiness_y']**2) for G in filtered_graphs[num_nodes]]
    algebraic_connectivity = [G.graph['algebraic_connectivity'] for G in filtered_graphs[num_nodes]]
    EGVL_RIG = [G.graph['EGVL_RIG'] for G in filtered_graphs[num_nodes]]
    
    # Scatter plot for all graphs
    # ax.scatter(edge_relation, spreadiness_norm, color='blue', label='Edge Relation')
    ax.scatter(algebraic_connectivity, spreadiness_norm, color='red', label='Algebraic Connectivity')
    
    # Filter graphs with EGVL_RIG over the threshold
    filtered_edge_relation = [er for er, rig in zip(edge_relation, EGVL_RIG) if rig > threshold]
    filtered_algebraic_connectivity = [ac for ac, rig in zip(algebraic_connectivity, EGVL_RIG) if rig > threshold]
    filtered_spreadiness_norm = [sn for sn, rig in zip(spreadiness_norm, EGVL_RIG) if rig > threshold]
    
    # Scatter plot for filtered graphs
    # ax.scatter(filtered_edge_relation, filtered_spreadiness_norm, color='red', label=f'EGVL_RIG > {threshold}')
    # ax.scatter(filtered_algebraic_connectivity, filtered_spreadiness_norm, color='green', label=f'EGVL_RIG > {threshold}')
    
    # same thing with rigid graphs
    edge_relation_rigid = [2 * G.graph['edge_connectivity'] * (1 - np.cos(np.pi / num_nodes)) for G in rigid_graphs if len(G.nodes) == num_nodes]
    algebraic_connectivity_rigid = [G.graph['algebraic_connectivity'] for G in rigid_graphs if len(G.nodes) == num_nodes]
    spreadiness_norm_rigid = [np.sqrt(G.graph['spreadiness_x']**2 + G.graph['spreadiness_y']**2) for G in rigid_graphs if len(G.nodes) == num_nodes]

    # Scatter plot for rigid graphs
    ax.scatter(algebraic_connectivity_rigid, spreadiness_norm_rigid, color='green', label='Algebraic Connectivity Rigid')
    
    ax.set_title(f'Number of Nodes: {num_nodes}')
    ax.set_xlabel('Connectivity')
    ax.set_ylabel('Spreadiness')
    ax.legend()

# Hide any unused subplots
for idx in range(len(range(min_agents, max_agents + 1)), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Define the thresholds
spreadiness_threshold = 7.5 
EGVL_rig_threshold = 0.1
algebraic_connectivity_threshold = 0.3

# Filter the graphs based on the thresholds
selected_graphs = [G for G in graphs if 
                   np.sqrt(G.graph['spreadiness_x']**2 + G.graph['spreadiness_y']**2) > spreadiness_threshold 
                   and G.graph['EGVL_RIG'] > EGVL_rig_threshold
                   and G.graph['algebraic_connectivity'] < algebraic_connectivity_threshold]

# Check if there are any selected graphs
if selected_graphs:
    # Plot the first selected graph
    G = selected_graphs[0]
    pos = nx.get_node_attributes(G, 'pos')
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=500, font_size=10)
    # Print the spreadiness and EGVL_RIG values on the plot
    spreadiness = np.sqrt(G.graph['spreadiness_x']**2 + G.graph['spreadiness_y']**2)
    EGVL_RIG = G.graph['EGVL_RIG']
    algebraic_connectivity = G.graph['algebraic_connectivity']
    plt.text(0.05, 0.95, f'Spreadiness: {spreadiness:.3f}\nConnectivity: {algebraic_connectivity:.3f}\nEGVL_RIG: {EGVL_RIG:.3f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    plt.show()
else:
    print("No graphs found with the specified criteria.")